In [1]:
import torch
import lightning.pytorch as ptl
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

import boda

# Set up

## Pick modules
Pick modules to define:
1. The data, how it's preprocessed and train/val/test split
2. The model, the architecture setup, loss function, etc.
3. The graph, how the data is used to train the model (i.e. training loop)

In [2]:
data_module = boda.data.SeqDataModule
model_module= boda.model.BassetBranched
graph_module= boda.graph.CNNBasicTraining

## Dummy dataset generation for testing purposes

In [3]:
import random
import csv

random.seed(42)

# Function to generate random DNA sequence
def generate_dna_sequence(length):
    return ''.join(random.choice('ACGT') for _ in range(length))

# Function to generate fake numerical score
def generate_numerical_score():
    return random.uniform(-4, 10)

# Number of sequences in the dataset
num_sequences = 200

# Length of DNA sequences
sequence_length = 200

## TRAIN
# Generating dummy dataset
dummy_train = []
for _ in range(num_sequences):
    sequence = generate_dna_sequence(sequence_length)
    score = generate_numerical_score()
    dummy_train.append((sequence, score))

# Save the dataset to a TSV file
traintsv_file = "dummy_train.tsv"
with open(traintsv_file, 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t')
    for sequence, score in dummy_train:
        writer.writerow([sequence, score])

## TEST
# Generating dummy dataset
dummy_test = []
for _ in range(num_sequences):
    sequence = generate_dna_sequence(sequence_length)
    score = generate_numerical_score()
    dummy_test.append((sequence, score))

# Save the dataset to a TSV file
testtsv_file = "dummy_test.tsv"
with open(testtsv_file, 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t')
    for sequence, score in dummy_test:
        writer.writerow([sequence, score])

## VALIDATE
# Generating dummy dataset
dummy_val = []
for _ in range(num_sequences):
    sequence = generate_dna_sequence(sequence_length)
    score = generate_numerical_score()
    dummy_val.append((sequence, score))

# Save the dataset to a TSV file
valtsv_file = "dummy_val.tsv"
with open(valtsv_file, 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t')
    for sequence, score in dummy_val:
        writer.writerow([sequence, score])

print(f"Dummy train with {num_sequences} sequences saved to '{traintsv_file}'.")
print(f"Dummy test with {num_sequences} sequences saved to '{testtsv_file}'.")
print(f"Dummy val with {num_sequences} sequences saved to '{valtsv_file}'.")


Dummy train with 200 sequences saved to 'dummy_train.tsv'.
Dummy test with 200 sequences saved to 'dummy_test.tsv'.
Dummy val with 200 sequences saved to 'dummy_val.tsv'.


## Initalize Data and Model
I added chr1 to test and chr2 to val to speed up this example. I also removed the reverse complmentat data augmentation.

In [4]:
data = data_module(
    train_file = "/home/ubuntu/boda2/analysis/AR001__rotation/dummy_train.tsv",
    test_file = "/home/ubuntu/boda2/analysis/AR001__rotation/dummy_test.tsv",
    val_file = "/home/ubuntu/boda2/analysis/AR001__rotation/dummy_val.tsv",
    right_flank = boda.common.constants.MPRA_DOWNSTREAM[:200],
    left_flank = boda.common.constants.MPRA_UPSTREAM[-200:]
)

model = model_module(
    n_outputs=2, 
    n_linear_layers=1, linear_channels=1000,
    linear_activation='ReLU', linear_dropout_p=0.12, 
    n_branched_layers=3, branched_channels=140, 
    branched_activation='ReLU', branched_dropout_p=0.56, 
    loss_criterion='L1KLmixed', kl_scale=5.0
)

## Append Graph to Model
Augment the model class to append functions from the graph module. A downside to this structure is that you need to make sure all relevent Graph args are defined (even if None is an acceptable default). This is because the `__init__` block in the Graph class doesn't run.

In [5]:
graph_args = {
    'optimizer': 'Adam', 
    'optimizer_args': {
        'lr': 0.0033, 'betas':[0.9, 0.999], 
        'weight_decay': 3.43e-4, 'amsgrad': True
    },
    'scheduler': 'CosineAnnealingWarmRestarts', 
    'scheduler_monitor': None, 
    'scheduler_interval': 'step',
    'scheduler_args': {
        'T_0': 4096,
    }
}

model.__class__ = type(
    'BODA_module',
    (model_module,graph_module),
    graph_args
)

In [6]:
model(torch.randn(10,4,600))

tensor([[-0.0234,  0.0481],
        [-0.0240,  0.0481],
        [-0.0236,  0.0481],
        [-0.0241,  0.0484],
        [-0.0239,  0.0481],
        [-0.0236,  0.0484],
        [-0.0240,  0.0481],
        [-0.0239,  0.0481],
        [-0.0242,  0.0476],
        [-0.0242,  0.0481]], grad_fn=<PermuteBackward0>)

## Lightning trainer
Normally we train for more epochs, but reduced in this example

In [7]:
checkpoint_callback = ModelCheckpoint(
    save_top_k=1, 
    monitor='prediction_mean_spearman', 
    mode='max'
)

stopping_callback = EarlyStopping(
    monitor='prediction_mean_spearman', 
    patience=5,
    mode='max'
)

trainer = ptl.Trainer(
    accelerator='gpu', devices=1, 
    min_epochs=5, max_epochs=20, 
    precision=16, callbacks= [
        checkpoint_callback,
        stopping_callback
    ]
)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## Train model

In [8]:
trainer.fit(model, data)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type           | Params
----------------------------------------------
0  | pad1      | ConstantPad1d  | 0     
1  | conv1     | Conv1dNorm     | 23.7 K
2  | pad2      | ConstantPad1d  | 0     
3  | conv2     | Conv1dNorm     | 660 K 
4  | pad3      | ConstantPad1d  | 0     
5  | conv3     | Conv1dNorm     | 280 K 
6  | pad4      | ConstantPad1d  | 0     
7  | maxpool_3 | MaxPool1d      | 0     
8  | maxpool_4 | MaxPool1d      | 0     
9  | linear1   | LinearNorm     | 2.6 M 
10 | branched  | BranchedLinear | 359 K 
11 | output    | GroupedLinear  | 282   
12 | nonlin    | ReLU           | 0     
13 | dropout   | Dropout        | 0     
14 | criterion | L1KLmixed      | 0     
----------------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
7.855     Total estimated model params size (MB)


Found 3927422 parameters


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,



----------------------------------------------------------------------------------------------------------------------------------------------------------
| current_epoch: 0.00000 | arithmetic_mean_loss: 7.13292 | harmonic_mean_loss: 21.26876 | prediction_mean_spearman: -0.08120 | entropy_spearman: 0.37293 |
----------------------------------------------------------------------------------------------------------------------------------------------------------



...


----------------------------------------------------------------------------------------------------------------------------------------------------------
| current_epoch: 9.00000 | arithmetic_mean_loss: 7.18579 | harmonic_mean_loss: 20.01982 | prediction_mean_spearman: 0.03595 | entropy_spearman: -0.16114 |
----------------------------------------------------------------------------------------------------------------------------------------------------------



Validation: 0it [00:00, ?it/s]


-----------------------------------------------------------------------------------------------------------------------------------------------------------
| current_epoch: 10.00000 | arithmetic_mean_loss: 7.12678 | harmonic_mean_loss: 19.05198 | prediction_mean_spearman: 0.12296 | entropy_spearman: -0.00148 |
-----------------------------------------------------------------------------------------------------------------------------------------------------------



Validation: 0it [00:00, ?it/s]


----------------------------------------------------------------------------------------------------------------------------------------------------------
| current_epoch: 11.00000 | arithmetic_mean_loss: 7.14193 | harmonic_mean_loss: 18.51028 | prediction_mean_spearman: 0.08296 | entropy_spearman: 0.07909 |
----------------------------------------------------------------------------------------------------------------------------------------------------------



In [9]:
import tempfile
import re
import sys
import os

def set_best(my_model, callbacks):
    """
    Set the best model checkpoint for the provided model.

    This function sets the state of the provided model to the state of the best checkpoint,
    as determined by the `ModelCheckpoint` callback.

    Args:
        my_model (nn.Module): The model to be updated.
        callbacks (dict): Dictionary of callbacks, including 'model_checkpoint'.

    Returns:
        nn.Module: The updated model.
    """
    with tempfile.TemporaryDirectory() as tmpdirname:
        try:
            best_path = callbacks['model_checkpoint'].best_model_path
            get_epoch = re.search('epoch=(\d*)', best_path).group(1)
            if 'gs://' in best_path:
                subprocess.call(['gsutil','cp',best_path,tmpdirname])
                best_path = os.path.join( tmpdirname, os.path.basename(best_path) )
            print(f'Best model stashed at: {best_path}', file=sys.stderr)
            print(f'Exists: {os.path.isfile(best_path)}', file=sys.stderr)
            ckpt = torch.load( best_path )
            my_model.load_state_dict( ckpt['state_dict'] )
            print(f'Setting model from epoch: {get_epoch}', file=sys.stderr)
        except KeyError:
            print('Setting most recent model', file=sys.stderr)
    return my_model

model = set_best(model, {'model_checkpoint': checkpoint_callback})

Best model stashed at: /home/ubuntu/boda2/analysis/AR001__rotation/lightning_logs/version_2/checkpoints/epoch=6-step=140.ckpt
Exists: True
Setting model from epoch: 6
